<a href="https://colab.research.google.com/github/Bevier19jac/3DDigitalDataset/blob/main/Face_Rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install face_recognition opencv-python
!pip install dlib
!pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=de2c9485b039450947cb0b430ade9582227e482a593a6412e55141aa863c022e
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [3]:
import face_recognition
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import drive


In [4]:
# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# Unzip known and unknown images
!unzip /content/drive/MyDrive/known.zip -d /content/known
!unzip /content/drive/MyDrive/unknown.zip -d /content/unknown


Archive:  /content/drive/MyDrive/known.zip
  inflating: /content/known/known/Donald Trump.jpg  
  inflating: /content/known/known/Aaron Peirsol.jpg  
  inflating: /content/known/known/Ben Affleck.jpg  
  inflating: /content/known/known/Adam Sandler.jpg  
  inflating: /content/known/known/Adam Scott.jpg  
  inflating: /content/known/known/Mike Pence.jpg  
Archive:  /content/drive/MyDrive/unknown.zip
  inflating: /content/unknown/unknown/Ben_Affleck_0007.jpg  
  inflating: /content/unknown/unknown/Zydrunas_Ilgauskas_0001.jpg  
  inflating: /content/unknown/unknown/Aaron_Peirsol_0003.jpg  
  inflating: /content/unknown/unknown/Adam_Sandler_0004.jpg  
  inflating: /content/unknown/unknown/Adam_Scott_0002.jpg  


In [6]:
# Path to known images
KNOWN_IMAGE_PATH = '/content/known/'

# Initialize list of known face encodings and their labels
known_face_encodings = []
known_face_names = []

# Iterate through the known images
for filename in os.listdir(KNOWN_IMAGE_PATH):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load image
        image = face_recognition.load_image_file(KNOWN_IMAGE_PATH + filename)

        # Get face encodings (only for the first face found in the image)
        face_encoding = face_recognition.face_encodings(image)[0]

        # Add the face encoding and the corresponding name
        known_face_encodings.append(face_encoding)
        known_face_names.append(filename.split('.')[0])  # Use filename as the label


In [7]:
# Path to unknown images
UNKNOWN_IMAGE_PATH = '/content/unknown/'

# Iterate through the unknown images
for filename in os.listdir(UNKNOWN_IMAGE_PATH):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the unknown image
        unknown_image = face_recognition.load_image_file(UNKNOWN_IMAGE_PATH + filename)

        # Find all face locations and encodings in the unknown image
        face_locations = face_recognition.face_locations(unknown_image)
        face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

        # Convert the image to BGR for OpenCV (since OpenCV uses BGR)
        unknown_image_bgr = cv2.cvtColor(unknown_image, cv2.COLOR_RGB2BGR)

        # Loop over each face found in the unknown image
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Check if the face matches any known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"  # Default label if no match is found

            # Find the known face with the smallest distance
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)

            # If there's a match, use the name of the best match
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a rectangle around the face
            cv2.rectangle(unknown_image_bgr, (left, top), (right, bottom), (0, 255, 0), 2)

            # Draw the name below the face
            cv2.rectangle(unknown_image_bgr, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(unknown_image_bgr, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2_imshow(unknown_image_bgr)


In [8]:
# Track missing people
missing_people = ["Mike Pence", "Donald Trump"]
for name in known_face_names:
    if name not in missing_people:
        missing_people.append(name)

print(f"Missing people: {missing_people}")


Missing people: ['Mike Pence', 'Donald Trump']
